In [ ]:
import torch
import torchvision
import glob
import matplotlib.pyplot as plt
import os

from torch.utils.data import DataLoader
from torch.utils.data import (
    Dataset,
)

from torchvision.datasets import ImageFolder
from torchvision.transforms import (
    RandomResizedCrop,
    RandomHorizontalFlip,
    ColorJitter,
    RandomGrayscale,
    RandomApply,
    Compose,
    GaussianBlur,
    ToTensor,
)
from PIL import Image

print(f'Torch-Version {torch.__version__}')
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'DEVICE: {DEVICE}')

#### **Source data preparation**

Prepapre the source domain dataset for self-supervised pre-training.

In [ ]:
def get_complete_transform(output_shape, kernel_size, s=1.0):
    """
    The color distortion transform.
    
    Args:
        s: Strength parameter.
    
    Returns:
        A color distortion transform.
    """
    rnd_crop = RandomResizedCrop(output_shape)
    rnd_flip = RandomHorizontalFlip(p=0.5)
    
    color_jitter = ColorJitter(0.8*s, 0.8*s, 0.8*s, 0.2*s)
    rnd_color_jitter = RandomApply([color_jitter], p=0.8)
    
    rnd_gray = RandomGrayscale(p=0.2)
    gaussian_blur = GaussianBlur(kernel_size=kernel_size)
    rnd_gaussian_blur = RandomApply([gaussian_blur], p=0.5)
    to_tensor = ToTensor()
    image_transform = Compose([
        to_tensor,
        rnd_crop,
        rnd_flip,
        rnd_color_jitter,
        rnd_gray,
        rnd_gaussian_blur,
    ])
    return image_transform


class ContrastiveLearningViewGenerator(object):
    """Take two random crops of one image as the query and key."""

    def __init__(self, base_transform, n_views=2):
        self.base_transform = base_transform
        self.n_views = n_views

    def __call__(self, x):
        views = [self.base_transform(x) for i in range(self.n_views)]
        return views

In [ ]:
class CustomDataset(Dataset):
    """Input Dataset"""

    def __init__(self, list_images, transform=None):
        """
        Args:
            list_images (list): List of all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.list_images = list_images
        self.transform = transform

    def __len__(self):
        return len(self.list_images)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        img_name = self.list_images[idx]
        image = Image.open(img_name).convert('RGB')  # Convert to RGB
        if self.transform:
            image = self.transform(image)

        return image

In [ ]:
# The size of the images
output_shape = [224,224]
kernel_size = [21,21] # 10% of the output_shape

# The custom transform
base_transforms = get_complete_transform(output_shape=output_shape, kernel_size=kernel_size, s=1.0)
custom_transform = ContrastiveLearningViewGenerator(base_transform=base_transforms)

# Load source dataset
sample_source_dataset = CustomDataset(
    list_images=glob.glob("sample_source_dataset/*/*.jpg"),
    transform=custom_transform
)

combined_dataset = sample_source_dataset

In [ ]:
# Plot a data sample with transformation 
plt.figure(figsize=(20,8))
def view_data(flowers, index):
    for i in range(1,6):
        images = flowers[index]
        view1, view2 = images
        plt.subplot(2, 5, i )
        plt.imshow(view1.permute(1,2,0))
        plt.subplot(2, 5, i +5)
        plt.imshow(view2.permute(1,2,0))

view_data(combined_dataset,300)

In [ ]:
# Set the batch size
BATCH_SIZE = 64

# Building the data loader
train_dl = torch.utils.data.DataLoader(
    combined_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    drop_last=True,
    pin_memory=True,
)

#### **Target domain data preparation**

Prepare the target domain dataset for domain-specific fine-tune and training.

In [ ]:
from torchvision.transforms import Resize

# Define the input data size
resize = Resize((224, 224))
ttensor = ToTensor()

# Resize and transform the target data sample into tensor 
custom_transform = Compose([
    resize,
    ttensor,
])

In [ ]:
# Set the input data path
sample_train_case = ImageFolder(
    root="target_dataset/train_images_directory/",
    transform=custom_transform
)

sample_test_case = ImageFolder(
    root="target_dataset/test_images_directory/",
    transform=custom_transform
)

# Set the batch size
BATCH_SIZE = 64

train_dataset = sample_train_case
test_dataset = sample_test_case

# Set the total class number of the target dataset
nu_train_classes = 3
nu_test_classes = 3

# Building the train data loader
train_dl = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,  # randomly shuffle the input data
    num_workers=os.cpu_count(),
    drop_last=False,  # drop the rest samples after last batch
    pin_memory=True,
)

# Building the test data loader
test_dl = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=os.cpu_count(),
    drop_last=False,
    pin_memory=True,
)